In [1]:
##Liberias

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import glob

In [2]:
#import tensorflow as tf
#from keras import layers
#import keras
#from datetime import datetime
#from sklearn.metrics import accuracy_score,classification_report

## 1. Datos

In [3]:
#Normas
DF_normas = pd.read_csv("normas_binary.csv",
                        index_col="Número de resolución",
                        )

#Tribunal
DF_tribunal = pd.read_csv("tribunal_binary.csv", 
                          index_col="Número de resolución")

#Empresas
DF_empresa = pd.read_csv("empresa_binary.csv", 
                         index_col="Número de resolución")


#
CA_WE = np.loadtxt("Criterios_emb.csv", delimiter=",")

#
CA_TFIDF = np.loadtxt("TF-IDF_Vectorization_Criterios.csv", 
                      delimiter=",")

TF_Todas = np.loadtxt("TF-IDF_Vectorization_Todas.csv", delimiter=",")

In [4]:
y = DF_normas.iloc[:, -1].values

In [5]:
normas_numpy = DF_normas.values[:, :-1]
tribunal_numpy = DF_tribunal.values[:, :-1]
empresa_numpy = DF_empresa.values[:, :-1]

## 2. Clasificación

In [6]:
def train_test_80_20(X, Y):
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=10)
    return X_train, X_test, Y_train, Y_test

In [7]:
def test_evaluation(model, X_test, y_test):
    from sklearn.metrics import classification_report
    
    # evaluación en test del modelo
    y_test_pred = model.predict(X_test)
    print(classification_report(y_test, y_test_pred))

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [9]:
# crear un iterador de CV para estandarizar los entrenamientos
# 5-fold cross-validation
kf = KFold(n_splits=5, random_state=0)

**Random Forest - empresa**
- bajo rendimiento

In [10]:
# optimización de hiperparámetros utilizando cross-validation

# MODELO: RF
# VARIABLES DE ENTRADA: empresa
# HYPERPARÁMETROS ÓPTIMOS: 

clf = RandomForestClassifier(n_estimators=100,
                             random_state=0
                             )

X = np.concatenate([TF_Todas], axis=1)
X_train, X_test, Y_train, Y_test = train_test_80_20(X, y)

scores = cross_val_score(clf, X_train, Y_train, 
                         cv=kf,
                         scoring="f1"
                        )

print(f"{scores.mean():.3f} +- {scores.std():.3f}")

0.776 +- 0.091


In [11]:
%time # pa que aprendas

# enternar el modelo con todos los datos de entrenamiento
clf.fit(X_train, Y_train)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


RandomForestClassifier(random_state=0)

In [12]:
# solo usar cuando ya hayas terminado lo de arriba
test_evaluation(clf, X_test, Y_test)

              precision    recall  f1-score   support

           0       0.89      0.55      0.68        31
           1       0.63      0.92      0.75        26

    accuracy                           0.72        57
   macro avg       0.76      0.74      0.71        57
weighted avg       0.77      0.72      0.71        57



**Random Forest - normas**

In [13]:
# optimización de hiperparámetros utilizando cross-validation

# MODELO: RF
# VARIABLES DE ENTRADA: normas
# HYPERPARÁMETROS ÓPTIMOS: 

clf = RandomForestClassifier(n_estimators=100, random_state=0)

X = np.concatenate([normas_numpy], axis=1)
X_train, X_test, Y_train, Y_test = train_test_80_20(X, y)

scores = cross_val_score(clf, X_train, Y_train, 
                         cv=kf,
                         scoring="f1"
                        )

print(f"{scores.mean():.3f} +- {scores.std():.3f}")

0.690 +- 0.063


In [14]:
%time
# enternar el modelo con todos los datos de entrenamiento
clf.fit(X_train, Y_train)

# solo usar cuando ya hayas terminado lo de arriba
test_evaluation(clf, X_test, Y_test)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs
              precision    recall  f1-score   support

           0       0.79      0.48      0.60        31
           1       0.58      0.85      0.69        26

    accuracy                           0.65        57
   macro avg       0.68      0.67      0.64        57
weighted avg       0.69      0.65      0.64        57



**Random Forest - criterios aplicables (TF-IDF)**

In [17]:
# optimización de hiperparámetros utilizando cross-validation

# MODELO: RF
# VARIABLES DE ENTRADA: CA (TF-IDF)
# HYPERPARÁMETROS ÓPTIMOS: 

clf = RandomForestClassifier(n_estimators=100, random_state=0)

X = np.concatenate([CA_TFIDF], axis=1)
X_train, X_test, Y_train, Y_test = train_test_80_20(X, y)

scores = cross_val_score(clf, X_train, Y_train, 
                         cv=kf,
                         scoring="f1"
                        )

print(f"{scores.mean():.3f} +- {scores.std():.3f}")

0.783 +- 0.060


In [18]:
%time
# enternar el modelo con todos los datos de entrenamiento
clf.fit(X_train, Y_train)

# solo usar cuando ya hayas terminado lo de arriba
test_evaluation(clf, X_test, Y_test)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs
              precision    recall  f1-score   support

           0       0.81      0.55      0.65        31
           1       0.61      0.85      0.71        26

    accuracy                           0.68        57
   macro avg       0.71      0.70      0.68        57
weighted avg       0.72      0.68      0.68        57



**Random Forest - normas + criterios aplicables (TF-IDF)**

In [21]:
# optimización de hiperparámetros utilizando cross-validation

# MODELO: RF
# VARIABLES DE ENTRADA: CA (normas + TF-IDF)
# HYPERPARÁMETROS ÓPTIMOS: 

clf = RandomForestClassifier(n_estimators=100, random_state=0)

X = np.concatenate([normas_numpy, CA_TFIDF], axis=1)
X_train, X_test, Y_train, Y_test = train_test_80_20(X, y)

scores = cross_val_score(clf, X_train, Y_train, 
                         cv=kf,
                         scoring="roc_auc"
                         )

print(f"{scores.mean():.3f} +- {scores.std():.3f}")

0.833 +- 0.072


In [22]:
%time
# enternar el modelo con todos los datos de entrenamiento
clf.fit(X_train, Y_train)

# solo usar cuando ya hayas terminado lo de arriba
test_evaluation(clf, X_test, Y_test)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs
              precision    recall  f1-score   support

           0       0.83      0.65      0.73        31
           1       0.67      0.85      0.75        26

    accuracy                           0.74        57
   macro avg       0.75      0.75      0.74        57
weighted avg       0.76      0.74      0.74        57

